# Imports

In [ ]:
import pandas as pd
import numpy as np
from autocorrect import Speller
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from ast import literal_eval

# Load raw data

In [ ]:
raw = pd.read_csv("data/raw_data.csv")
raw.head()

# Adjust columns

In [ ]:
# rename some columns
df = raw.copy()
df = df.rename(columns={"Experience Tags":"Experience","Can Help With":"Skills"})

# Parse keywords

In [ ]:
experience_tags = [
    "Enterprise",
    "Healthcare",
    "Marketplace",
    "Academia",
    "Analytics",
    "Biotech",
    "Ecommerce",
    "FinTech",
    "Future of Work",
    "Future of Food",
    "Insurance",
    "Science",
    "Blockchain/Crypto",
    "Direct to Consumer",
    "Consumer",
    "AI/ML",
    "B2B",
    "Climate",
    "Community",
    "Government",
    "SaaS",
    "Ecommerce",
    "Coaching",
    "Developer Tools",
    "Clean Technology",
    "IOT",
    "Productivity",
    "Real Estate",
    "Social",
    "Mental Health/Wellness",
    "Talent",
    "Hardware",
    "Education",
    "No-Code",
    "Gaming",
    "Transportation & Travel",
    "Venture",
    "Robotics",
    "People Ops",
    "VR/AR",
    "Drones",
    "Security",
    "Legal",
    "Social Impact",
    "Construction",
]

In [ ]:
skill_tags = [
    "Software Engineering",
    "Business Development",
    "Product Management",
    "Research",
    "Communications",
    "Data Science",
    "Operations",
    "Growth",
    "Analytics",
    "Marketing",
    "Product Management",
    "Recruiting",
    "C-Suite Executives",
    "Sales",
    "Design",
    "Customer Service",
    "Finance",
    "Hardware Engineering",
]

In [ ]:
stage_tags = [
    "Open to new ideas",
    "In early stages of exploring a specific idea",
    "Raising funding",
    "Built a basic prototype/MVP",
    "Starting to onboard customers",
    "Seed+"
]

In [ ]:
objective_tags = [
    "Looking for co-founder",
    "Looking to join another team",
    "Looking for first employees"
]

In [ ]:
interest_tags = experience_tags

In [ ]:
looking_for_tags = skill_tags

In [ ]:
all_tags = {
    "Experience": experience_tags,
    "Skills": skill_tags,
    "Stages": stage_tags,
    "Objectives": objective_tags,
    "Interests": interest_tags,
    "Looking for": looking_for_tags
}

In [ ]:
# split into tokens/phrases
cols_to_parse = ["Experience", "Skills", "Stages", "Objectives", "Interests", "Looking for"]
for idx, row in df.iterrows():
    for col in cols_to_parse:
        try:
            if isinstance(row[col],str) and row[col] != "NaN":
                row[col] = list(np.unique([tag for tag in all_tags[col] if tag in row[col]]))
        except:
            print(f"error in row {idx}, column {col}")
df

# Save cleaned data

In [ ]:
# cast everything as strings
df = df.replace(np.nan, "NaN", regex=True)

In [ ]:
df.to_csv("data/clean_data.csv",index=False)

# Save all responses

In [ ]:
df = df.astype(str)

In [ ]:
cols_to_include = ["Experience","Skills","Interests","Stages","Objectives","Looking for"]
all_responses = {"Name":[],"Category":[],"Response":[]}
for idx,row in df.iterrows():
    name = row["Name"]
    for col in cols_to_include:
        if row[col] != "nan" and row[col] != "NaN":
            responses = literal_eval(row[col])
            for resp in responses:
                all_responses["Name"].append(name)
                all_responses["Category"].append(col)
                all_responses["Response"].append(resp)
all_responses = pd.DataFrame.from_dict(all_responses)
all_responses

In [ ]:
all_responses.to_csv("data/response_data.csv",index=False)

# Replace phrases with keywords

In [ ]:
phrase_map = {
    "Open to new ideas": "ideas",
    "In early stages of exploring a specific idea": "exploring",
    "Raising funding": "funding",
    "Built a basic prototype/MVP": "prototype",
    "Starting to onboard customers": "customers",
    "Seed+": "seed",
    "Looking for co-founder": "partner",
    "Looking to join another team": "team",
    "Looking for first employees": "employees"
}

cols = ["Stages","Objectives"]
for idx, row in df.iterrows():
    for col in cols:
        try:
            if isinstance(row[col],str) and row[col] != "nan" and row[col] != "NaN":
                row[col] = [phrase_map[tag] if tag in phrase_map else tag for tag in literal_eval(row[col])]
        except:
            print(f"error in row {idx}, column {col}")
df

# Convert/remove acronyms/jargon

In [ ]:
jargon_map = {
    "AI": "Artificial Intelligence",
    "ML": "Machine Learning",
    "B2B": "Business to Business",
    "SaaS": "Software as a Service",
    "FinTech": "Financial Technology",
    "Biotech": "Biotechnology",
    "Crypto": "Cryptocurrency",
    "Ops": "Operations",
    "VR/AR": "Virtual Reality and Artificial Reality",
    "IOT": "Internet of Things",
    "MVP": "minimum viable product",
    "C-Suite": ""
}

In [ ]:
for idx, row in df.iterrows():
    for col in cols_to_parse:
        try:
            if isinstance(row[col],str) and row[col]!="nan" and row[col]!="NaN":
                tags = [tag.replace("/"," and ") for tag in literal_eval(row[col])]
                row[col] = [jargon_map[term] if term in jargon_map else term for tag in tags for term in tag.split()]   
        except:
            print(f"error in row {idx}, column {col}")
df

# Process data
- tokenize
- remove non-letters
- convert to lowercase
- spell-check

In [ ]:
spell = Speller()
stop_words = np.unique(stopwords.words('english') + ["the"])
for idx, row in df.iterrows():
    for col in cols_to_parse:
        try:
            if isinstance(row[col],list):
                tokens = wordpunct_tokenize(" ".join([term.lower() for term in row[col]]))
                row[col] = [spell(token) for token in tokens if token.isalpha() and token not in stop_words]
        except:
            print(f"error in row {idx}, column {col}")
df

# Save processed data

In [ ]:
df.to_csv("data/processed_data.csv",index=False)